In [1]:
import pandas as pd
import mysql.connector as mc
import matplotlib
import calendar

In [2]:
db = mc.connect(
  host='172.18.55.6',
  user='comandato',
  passwd='comandato123',
)
cur= db.cursor()

In [3]:
Dia='23'
Mes='01'
Anio='2019'
fecha_pago=Anio+Mes+Dia

In [4]:
F_Mes='ENERO'

# OBTENER GESTION DE ASESOR

### CONVENIO

In [5]:
querry_gestion_cuotas="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.GESTION where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00' AND COMENTARIOS_GESTIONO LIKE '%CAMPAÑA CONVENIO%';""".format(Anio,Mes,Dia)
gestion_cuotas=pd.read_sql(querry_gestion_cuotas, con=db)

In [6]:
gestion_asesor_cuotas=gestion_cuotas[(gestion_cuotas['ACCION_REALIZADA']=='HACER LLAMADA') & (gestion_cuotas['CONTACTO_GESTIONO']=='CLIENTE')& (gestion_cuotas['RESPUESTA_OBTENIDA']!='YA PAGO')]

In [7]:
gestion_asesor_cuotas=gestion_asesor_cuotas.sort_values(by=['FECHA_GESTION'],ascending=[True])
gestion_asesor_cuotas=gestion_asesor_cuotas.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')
gestion_asesor_cuotas = gestion_asesor_cuotas.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'IDENTIFICACION'})

### CORRIENTE

In [8]:
querry_gestion="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.GESTION where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00' AND COMENTARIOS_GESTIONO NOT LIKE '%CAMPAÑA CONVENIO%';""".format(Anio,Mes,Dia)
gestion=pd.read_sql(querry_gestion, con=db)

In [9]:
gestion_asesor=gestion[(gestion['ACCION_REALIZADA']=='HACER LLAMADA') & ((gestion['CONTACTO_GESTIONO']=='CLIENTE') | (gestion['CONTACTO_GESTIONO']=='TERC. VALIDO'))& (gestion['RESPUESTA_OBTENIDA']!='YA PAGO')]

In [10]:
gestion_asesor=gestion_asesor.sort_values(by=['FECHA_GESTION'],ascending=[True])

In [11]:
gestion_asesor=gestion_asesor.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')

In [12]:
gestion_asesor.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
2,0927337402,D013102,2,TERC. VALIDO,HACER LLAMADA,MENSAJE CON TERCERO,0990967830
1129,0914452701,D013112,2,CLIENTE,HACER LLAMADA,RECLAMO VENDIO VEHICULO,0989656647
1134,0925845497,D013116,2,TERC. VALIDO,HACER LLAMADA,MENSAJE CON TERCERO,0990670726
1136,0914701388,D013116,2,CLIENTE,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0980776119
1138,0914450390,D013142,2,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0980072998


# OBTENER GESTION IVR

In [13]:
querry_masivos="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.gestion_ivr where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00';""".format(Anio,Mes,Dia)
gestion_masivos=pd.read_sql(querry_masivos, con=db)

In [14]:
gestion_ivr=gestion_masivos[(gestion_masivos['ACCION_REALIZADA']=='ENVIAR IVR') & (gestion_masivos['RESPUESTA_OBTENIDA']=='CONTESTA IVR')]

In [15]:
gestion_ivr.tail()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
326020,0924698426,D05554,10,TITULAR,ENVIAR IVR,CONTESTA IVR,043077376
326021,0924698582,D05554,10,TITULAR,ENVIAR IVR,CONTESTA IVR,0994477636
326022,0924699077,D05554,10,TITULAR,ENVIAR IVR,CONTESTA IVR,0988914690
326023,0924708357,D05554,10,TITULAR,ENVIAR IVR,CONTESTA IVR,042487810
326024,0924708431,D05554,10,TITULAR,ENVIAR IVR,CONTESTA IVR,0986532533


# OBTENER GESTION SMS

In [16]:
gestion_sms=gestion_masivos[gestion_masivos['ACCION_REALIZADA']=='ENVIAR SMS']

In [17]:
gestion_sms.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
109527,1002339149,D05554,2,TITULAR,ENVIAR SMS,SMS ENVIADO,0991965227
109528,0101258317,D05554,2,TITULAR,ENVIAR SMS,SMS ENVIADO,0990565513
109529,0101602258,D05554,2,TITULAR,ENVIAR SMS,SMS ENVIADO,0990073890
109530,0102107851,D05554,2,TITULAR,ENVIAR SMS,SMS ENVIADO,0968112290
109531,0102229119,D05554,2,TITULAR,ENVIAR SMS,SMS ENVIADO,0999944647


In [18]:
gestiones=[gestion_asesor,gestion_ivr,gestion_sms]
total_gestiones=pd.concat(gestiones)

In [19]:
total_gestiones.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
2,0927337402,D013102,2,TERC. VALIDO,HACER LLAMADA,MENSAJE CON TERCERO,0990967830
1129,0914452701,D013112,2,CLIENTE,HACER LLAMADA,RECLAMO VENDIO VEHICULO,0989656647
1134,0925845497,D013116,2,TERC. VALIDO,HACER LLAMADA,MENSAJE CON TERCERO,0990670726
1136,0914701388,D013116,2,CLIENTE,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0980776119
1138,0914450390,D013142,2,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0980072998


In [20]:
total_gestiones=total_gestiones.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')

In [21]:
total_gestiones = total_gestiones.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'IDENTIFICACION'})

In [22]:
total_gestiones.head()

,IDENTIFICACION,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
2,0927337402,D013102,2,TERC. VALIDO,HACER LLAMADA,MENSAJE CON TERCERO,0990967830
1129,0914452701,D013112,2,CLIENTE,HACER LLAMADA,RECLAMO VENDIO VEHICULO,0989656647
1134,0925845497,D013116,2,TERC. VALIDO,HACER LLAMADA,MENSAJE CON TERCERO,0990670726
1136,0914701388,D013116,2,CLIENTE,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0980776119
1138,0914450390,D013142,2,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0980072998


# MULTAS ASIGNADAS

In [23]:
multas="""SELECT *   FROM SISTEMECUADOR_ATM.MULTAS_MENSUALES where Mes={0}""".format(Mes)
resultado_multas=pd.read_sql(multas, con=db)

In [24]:
resultado_multas.head()

,Cedula,No_multa,Anio,Mes,Dia,Dia_retiro,Dias_mora,Saldo,Cartera,Segmento,Descripcion,Placa,Pago,dia_pago,tipo
0,00042448K,5722394,2019,1,2,None,72,115.80,CORRIENTE,MULTAS NUEVAS,COND.EXCEDA DENTRO RANGO MODERADO LÍMITES DE V...,GRP0269,None,None,None
1,100025030,4929258,2019,1,2,None,249,19.30,MENOR,MULTAS NUEVAS,COND. QUE NO UTILICE CINTURON DE SEGURIDAD.,nan,None,None,None
2,1000559797,4171791,2019,1,2,None,429,56.25,CORRIENTE,MULTAS NUEVAS,DOBLE COLUMNA / ESTACIONAR SITIOS PROHIB. FUER...,LBG0289,None,None,None
3,1000559797,4171793,2019,1,2,None,429,56.25,CORRIENTE,MULTAS NUEVAS,DOBLE COLUMNA / ESTACIONAR SITIOS PROHIB. FUER...,LBG0289,None,None,None
4,1000559797,5433481,2019,1,2,None,138,19.30,CORRIENTE,MULTAS NUEVAS,COND. QUE NO UTILICE CINTURON DE SEGURIDAD.,nan,None,None,None


In [25]:
dias_mora_multa=resultado_multas[['No_multa','Dias_mora']].copy()

In [26]:
dias_mora_multa = dias_mora_multa.rename(columns={'No_multa':'ID_FACTURA','Dias_mora':'DIAS_MORA'})
dias_mora_multa['ID_FACTURA']=dias_mora_multa['ID_FACTURA'].apply(int)

In [27]:
dias_mora_multa.head()

,ID_FACTURA,DIAS_MORA
0,5722394,72
1,4929258,249
2,4171791,429
3,4171793,429
4,5433481,138


# PAGOS

In [28]:
pago_corr={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\Sistema de cobro - '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CORRIENTE',
    'cartera':'CORRIENTE',
    'identificador':'IDENTIFICACION'
}

pago_cuot={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\\Sistema de cobro - '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CUOTA_CERO',
    'cartera':'CUOTA',
    'identificador':'ID'
}

pago_conv={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\\Sistema de cobro - '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CONVENIOS',
    'cartera':'CONVENIO',
    'identificador':'ID'
}

In [29]:
pagos_corr=pd.read_excel(open(pago_corr['ruta'],'rb'), sheet_name=pago_corr['hoja'], dtype={pago_corr['identificador']: str})
pagos_cuot=pd.read_excel(open(pago_cuot['ruta'],'rb'), sheet_name=pago_cuot['hoja'], dtype={pago_cuot['identificador']: str})
pagos_conv=pd.read_excel(open(pago_conv['ruta'],'rb'), sheet_name=pago_conv['hoja'], dtype={pago_conv['identificador']: str})

In [30]:
def agregar_ceros(base):
    ced=base['ID']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

In [31]:
pagos_cuot['ID']=pagos_cuot.apply(agregar_ceros,axis=1)

### CORRIENTE

In [32]:
def agregar_ceros2(base):
    ced=base['IDENTIFICACION']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

In [33]:
pagos_corr['IDENTIFICACION']=pagos_corr.apply(agregar_ceros2,axis=1)

In [34]:
def tipo(base):
    casa=base['CASA_COBRO']
    if casa=='SISTEMA SALDO MENORES':
        return 'MENORES'
    else:
        return 'CORRIENTE'

In [35]:
pagos_corr['DIA_PAGO'] = pagos_corr['FECHA_PAGO'].dt.day
pagos_corr = pagos_corr.rename(columns={'CASA COBRO':'CASA_COBRO'})
pagos_corr['TIPO']=pagos_corr.apply(tipo,axis=1)
cruce_pag_corr=pd.merge(pagos_corr,dias_mora_multa,on=['ID_FACTURA'],how='left')
cruce_pag_corr=cruce_pag_corr[['IDENTIFICACION','ID_FACTURA','VALOR_INFRACCION','FECHA_PAGO','DIA_PAGO','TIPO','DIAS_MORA']]
cruce_pag_corr.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0920819265,5297974,57.9,2019-01-02,2,CORRIENTE,166
1,0908634785,4294228,37.5,2019-01-02,2,CORRIENTE,399
2,0909391609,5375532,57.9,2019-01-02,2,CORRIENTE,159
3,0930881529,5107022,38.6,2019-01-02,2,CORRIENTE,208
4,0922490297,5436594,19.3,2019-01-02,2,CORRIENTE,138


## MENORES

In [36]:
def identificar_pago(base):
    valor1 = base['VALOR_INFRACCION']
    valor2 = base['VALOR_PAGO_CAPITAL']
    mora = base['DIAS_MORA']
    if mora<=180:
        return valor2
    return valor1

In [37]:
pagos_cuot['TIPO']='CUOTA'
pagos_cuot['DIA_PAGO'] = pagos_cuot['FECHA_RE'].dt.day
pagos_cuot['MONTO_PAGO']=pagos_cuot.apply(identificar_pago,axis=1)
pagos_cuot_resumen=pagos_cuot[['ID','ID_FACTURA_INICIAL','MONTO_PAGO','FECHA_RE','DIA_PAGO','TIPO']]
pagos_cuot_resumen = pagos_cuot_resumen.rename(columns={'ID':'IDENTIFICACION','ID_FACTURA_INICIAL':'ID_FACTURA','MONTO_PAGO':'VALOR_INFRACCION','FECHA_RE':'FECHA_PAGO'})
cruce_pag_cuot=pd.merge(pagos_cuot_resumen,dias_mora_multa,on=['ID_FACTURA'],how='left')
cruce_pag_cuot.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0800467276,5272640,23.16,2019-01-02 15:54:48,2,CUOTA,181
1,0800467276,5338357,23.16,2019-01-02 15:54:48,2,CUOTA,160
2,0912757234,3190025,18.75,2019-01-03 12:20:49,3,CUOTA,637
3,0912757234,3456959,112.50,2019-01-03 12:20:49,3,CUOTA,569
4,0912757234,3543397,37.50,2019-01-03 12:21:58,3,CUOTA,547


# CONVENIO

In [38]:
def agregar_ceros3(base):
    ced=base['ID']
    if len(ced)==9 or len(ced)==12:
        if ced.isdigit():
            return '0'+ced
    return ced

In [39]:
pagos_conv['TIPO']='CONVENIO'
pagos_conv['DIA_PAGO'] = pagos_conv['FECHA_PAGO'].dt.day

In [40]:
pagos_conv['ID']=pagos_conv.apply(agregar_ceros3,axis=1)

In [41]:
pagos_conv.head()

,ID,ID_CONVENIO,NRO_CUOTA_EMISION,VALOR_PAGAR_CUOTA,FECHA_PAGO,FECHA_VENCIMIENTO,CASA COBRO,TIPO,DIA_PAGO
0,0909148181,277203,6,64.99,2019-01-02,2018-03-26,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,2
1,0914931555,335149,1,66.76,2019-01-02,2018-11-19,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,2
2,0903951655,336932,2,101.20,2019-01-02,2018-12-31,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,2
3,1703473502,336950,2,30.82,2019-01-02,2018-12-31,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,2
4,0916634793,337076,2,30.88,2019-01-02,2018-12-31,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,2


In [42]:
pagos_conv_resumen=pagos_conv[['ID','ID_CONVENIO','VALOR_PAGAR_CUOTA','FECHA_PAGO','DIA_PAGO','TIPO','NRO_CUOTA_EMISION']]
pagos_conv_resumen = pagos_conv_resumen.rename(columns={'ID':'IDENTIFICACION','ID_CONVENIO':'ID_FACTURA','VALOR_PAGAR_CUOTA':'VALOR_INFRACCION','NRO_CUOTA_EMISION':'DIAS_MORA'})

cruce_pag_conv=pd.merge(pagos_conv_resumen,gestion_asesor_cuotas,on=['IDENTIFICACION'],how='left')

## UNIR LOS PAGOS

In [43]:
pagos=[cruce_pag_cuot,cruce_pag_corr]
total_pagos=pd.concat(pagos)

In [44]:
total_pagos.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0800467276,5272640,23.16,2019-01-02 15:54:48,2,CUOTA,181
1,0800467276,5338357,23.16,2019-01-02 15:54:48,2,CUOTA,160
2,0912757234,3190025,18.75,2019-01-03 12:20:49,3,CUOTA,637
3,0912757234,3456959,112.50,2019-01-03 12:20:49,3,CUOTA,569
4,0912757234,3543397,37.50,2019-01-03 12:21:58,3,CUOTA,547


In [45]:
def honorarios(base):
    tipo=base['TIPO']
    mora=base['DIAS_MORA']
    valor=base['VALOR_INFRACCION']
    if tipo=='CONVENIO':
        return valor*0.035
    if tipo=='CUOTA'and mora>180:
        return valor*0.05
    if mora<61:
        return valor*0.02
    if mora<181:
        return valor*0.04
    if mora<361:
        return valor*0.05
    if mora>360:
        return valor*0.06
    return 0.0

In [46]:
total_pagos['HONORARIOS']=total_pagos.apply(honorarios,axis=1)
total_pagos.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,HONORARIOS
0,0800467276,5272640,23.16,2019-01-02 15:54:48,2,CUOTA,181,1.1580
1,0800467276,5338357,23.16,2019-01-02 15:54:48,2,CUOTA,160,0.9264
2,0912757234,3190025,18.75,2019-01-03 12:20:49,3,CUOTA,637,0.9375
3,0912757234,3456959,112.50,2019-01-03 12:20:49,3,CUOTA,569,5.6250
4,0912757234,3543397,37.50,2019-01-03 12:21:58,3,CUOTA,547,1.8750


## CRUCE CON GESTIONES

In [47]:
cruce_honorario=pd.merge(total_pagos,total_gestiones,on=['IDENTIFICACION'],how='left')

In [48]:
cruce_honorario.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,HONORARIOS,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
0,0800467276,5272640,23.16,2019-01-02 15:54:48,2,CUOTA,181,1.1580,D05554,2.0,TITULAR,ENVIAR IVR,CONTESTA IVR,042676803
1,0800467276,5338357,23.16,2019-01-02 15:54:48,2,CUOTA,160,0.9264,D05554,2.0,TITULAR,ENVIAR IVR,CONTESTA IVR,042676803
2,0912757234,3190025,18.75,2019-01-03 12:20:49,3,CUOTA,637,0.9375,D013112,4.0,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0984908378
3,0912757234,3456959,112.50,2019-01-03 12:20:49,3,CUOTA,569,5.6250,D013112,4.0,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0984908378
4,0912757234,3543397,37.50,2019-01-03 12:21:58,3,CUOTA,547,1.8750,D013112,4.0,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0984908378


In [49]:
def validar(base):
    dia_pago=base['DIA_PAGO']
    dia_gestion=base['FECHA_GESTION']
    if dia_gestion<=dia_pago:
        return 'SI'
    return 'NO'

In [50]:
cruce_honorario['VALIDO']=cruce_honorario.apply(validar,axis=1)
cruce_honorario.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,HONORARIOS,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION,VALIDO
0,0800467276,5272640,23.16,2019-01-02 15:54:48,2,CUOTA,181,1.1580,D05554,2.0,TITULAR,ENVIAR IVR,CONTESTA IVR,042676803,SI
1,0800467276,5338357,23.16,2019-01-02 15:54:48,2,CUOTA,160,0.9264,D05554,2.0,TITULAR,ENVIAR IVR,CONTESTA IVR,042676803,SI
2,0912757234,3190025,18.75,2019-01-03 12:20:49,3,CUOTA,637,0.9375,D013112,4.0,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0984908378,NO
3,0912757234,3456959,112.50,2019-01-03 12:20:49,3,CUOTA,569,5.6250,D013112,4.0,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0984908378,NO
4,0912757234,3543397,37.50,2019-01-03 12:21:58,3,CUOTA,547,1.8750,D013112,4.0,CLIENTE,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0984908378,NO


In [51]:
cruce_pag_conv['HONORARIOS']=cruce_pag_conv.apply(honorarios,axis=1)
cruce_pag_conv.head()
cruce_pag_conv['VALIDO']=cruce_pag_conv.apply(validar,axis=1)
cruce_pag_conv.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION,HONORARIOS,VALIDO
0,0909148181,277203,64.99,2019-01-02,2,CONVENIO,6,NaN,NaN,NaN,NaN,NaN,NaN,2.27465,NO
1,0914931555,335149,66.76,2019-01-02,2,CONVENIO,1,NaN,NaN,NaN,NaN,NaN,NaN,2.33660,NO
2,0903951655,336932,101.20,2019-01-02,2,CONVENIO,2,NaN,NaN,NaN,NaN,NaN,NaN,3.54200,NO
3,1703473502,336950,30.82,2019-01-02,2,CONVENIO,2,NaN,NaN,NaN,NaN,NaN,NaN,1.07870,NO
4,0916634793,337076,30.88,2019-01-02,2,CONVENIO,2,NaN,NaN,NaN,NaN,NaN,NaN,1.08080,NO


In [52]:
pagos_total=[cruce_honorario,cruce_pag_conv]
total_pagos_all=pd.concat(pagos_total)

c:\users\michael\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


## HONORARIOS VALIDOS

In [53]:
print(total_pagos_all[total_pagos_all['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(total_pagos_all[total_pagos_all['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

           VALOR_INFRACCION  HONORARIOS
TIPO                                   
CONVENIO            2552.51    89.33785
CORRIENTE          89984.50  3455.83000
CUOTA              49514.22  2294.18090
VALOR_INFRACCION    142051.23000
HONORARIOS            5839.34875
dtype: float64


In [54]:
total_pagos_all[total_pagos_all['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).plot.line()

## HONORARIOS TOTALES

In [55]:
print(total_pagos_all.groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(total_pagos_all.groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

           VALOR_INFRACCION  HONORARIOS
TIPO                                   
CONVENIO           46351.35  1622.29725
CORRIENTE         103512.40  3950.50900
CUOTA              52921.52  2427.75190
VALOR_INFRACCION    202785.27000
HONORARIOS            8000.55815
dtype: float64


## HONORARIOS NO VALIDOS

In [56]:
print(total_pagos_all[total_pagos_all['VALIDO']=='NO'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(total_pagos_all[total_pagos_all['VALIDO']=='NO'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

           VALOR_INFRACCION  HONORARIOS
TIPO                                   
CONVENIO           43798.84   1532.9594
CORRIENTE          13527.90    494.6790
CUOTA               3407.30    133.5710
VALOR_INFRACCION    60734.0400
HONORARIOS           2161.2094
dtype: float64


In [57]:
nuevo_archivo='ANALISIS DE PAGOS.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
total_pagos_all.to_excel(writer, sheet_name='GESTION TOTAL')
worksheet = writer.sheets['GESTION TOTAL']
writer.save()

In [58]:
nuevo_archivo='GESTION ASESOR.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
gestion_asesor.to_excel(writer, sheet_name='GESTION TOTAL')
worksheet = writer.sheets['GESTION TOTAL']
writer.save()